<a id="top"></a>
#### QCT. Practices with Adiabatic Quantum Computing

# Code Assignment. Nurse Scheduling Problem

#### Lecturers:
- Alejandro Borrallo Rentero, a.a.borrallo.rentero@accenture.com
- Rafael Martín-Cuevas Redondo, rafamartinc@pm.me
- Víctor Pérez Fernández, v.perez.fernandez@accenture.com

***

In this notebook, you will implement one instance of the Nurse Scheduling Problem, introduced by Ikeda, K., Nakamura, Y. & Humble, T.S at "Application of Quantum Annealing to Nurse Scheduling Problem". Sci Rep 9, 12837 (2019). https://doi.org/10.1038/s41598-019-49172-3

During the following sections, you will have to implement the three constraints of this problem (whose mathematical formulation is provided and explained on each section), and adjust the Lagrange multipliers to fit the largest dataset being considered (```nurse_scheduling_large.yml```). The pre-processing, post-processing, basic initialization of the BQM and main structure of the code are provided.

### Table of Contents

1. [Account configuration](#account_configuration)
2. [Two-Shift System with Shift and Nurse Constraints](#nurse_scheduling_1)

    - [Pre-processing](#ns1_preprocessing)
    - [Preparing the model](#ns1_model_preparation)
    - [Add the soft nurse constraint](#ns1_soft_nurse_constraint)
    - [Add the hard shift constraint](#ns1_hard_shift_constraint)
    - [Add the hard nurse constraint](#ns1_hard_nurse_constraint)
    - [Post-processing](#ns1_postprocessing)
    - [Execution and results](#ns1_execution)
    
    
    
4. [Sources and additional references](#ref)

The following block will try to import everything we need to run this notebook. If any of the import statements fails, please make sure that your Python environment has every pip package mentioned in the [requirements.txt](/edit/requirements.txt) file provided with this notebook.

The way to check or install a package differs from system to system, but usually you can check all the packages installed in your system with ```pip freeze``` and you can install all libraries needed for this exercise with ```pip install -r requirements.txt``` and the requirements.txt file mentioned above.

In [78]:
import os
import yaml
import dimod
import neal

print('Modules imported successfully!')

Modules imported successfully!


[Back to the top](#top)

***
<a id="account_configuration"></a>

## 1. Account configuration

In this section, we will log into our account at https://cloud.dwavesys.com/leap/ to retrieve our API Token and save it in a file named 'dwave_token.txt', located on the same directory as this Jupyter notebook. We will then be able to retrieve the backends that are available to us.

In [79]:
with open('dwave_token.txt', 'r') as file:
    os.environ['DWAVE_API_TOKEN'] = file.read()

[Back to the top](#top)

***
<a id="nurse_scheduling_1"></a>

## 2. Two-Shift System with Shift and Nurse Constraints

In this section, we will implement a simple version of the Nurse Scheduling Problem, described in the referenced article and defined as follows (albeit a few minor modifications, applied to simplify the formulation of the problem):

Let us consider $N$ nurses (mapped to the variable ```ǹum_nurses```), each one identified through a index $n$ (```nurse```). We want to assign the nurses to several working days from a total number of $D$ (```num_days```) to be covered, each one identified through a index $d$ (```day```). Our binary variables for this problem can be defined as $x_{n,d}$, referencing whether nurse $n$ will work on day $d$ or not. We will consider the following requirements to define our Hamiltonian:

- Each nurse will be assigned a number of working days $F_n$ (```ẁorking_days[nurse]```) that may vary from nurse to nurse, out of the total number of days available $D$. Presumably, $F_n \leq D$, leaving a number of days off as a consequence. This is described in the paper as the **soft nurse constraint**. We have removed the considerations about nurse preferences from the original paper.


- Each day, identified by the index $d$ (```day```) will have an associated workload $W_d$ (```workload[day]```) that may vary from day to day and will have to be assumed by one or several nurses. Each nurse will have an associated level of effort $E_n$ (```effort_level[nurse]```) that varies from nurse to nurse, so that the sum of the effort levels of the nurses assigned to any given date is at least equal to that day's workload. This is described in the paper as the **hard shift constraint**.


- Additionally, we want to define relations between different days and/or nurses. To do that, we will use a coefficient $J_{n,d,n',d'}$ (```penalty[nurse_1][nurse_2][day_1][day_2]```), that can act as a bonus or penalty between nurse $n$ working on day $d$ and nurse $n'$ working on day $d'$. To illustrate this possibility, we will try to avoid having the same nurse working two days in a row, by assigning a high penalty if this term is considering the same nurse ($n = n'$) over consecutive days ($d' = d + 1$). This is described in the paper as the **hard nurse constraint**.

In this scenario, there is not a clear objective function: we will have to set up the strength of each constraint correctly so that the solutions retrieved from the solver prioritize the constraints as desired. Presumably, the Lagrange multiplers of the hard constraints will be heavier than the ones associated to the soft constraint. We will now build our model considering each constraint independently.

[Back to the top](#top)

<a id="ns1_preprocessing"></a>
### 2.1. Pre-processing

We start our implementation by gathering all the data available to us: the number of nurses and days, and the parameters of our problem, detailed previously.

In [80]:
def get_problem_data(path):
    """
    Retrieves the definition of the problem and returns all associated data.
    
    :param path: String containing the full path to the file that should be loaded.
    :return: Tuple containing the following three elements:
        - working_days: List containing the number of working days agreed with each
            nurse, for every nurse (len(working_days) == num_nurses). E.g: [1, 1, 2, 2].
        - effort_level: List containing the effort level that each nurse can deliver,
            to try to reach the total workload required for each day. E.g: [1, 4, 2, 3].
        - workload: List containing the workload associated to each day, for every
            available day (len(workload) == num_days). E.g.: [5, 5, 5].
    """
    working_days = None  # Number of working days agreed with each nurse.
    effort_level = None  # Workload that each nurse can assume.
    workload = None      # Total workload for each day.
    
    print('Loading file: {}'.format(path))

    with open(path, 'r') as ymlfile:
        data = yaml.load(ymlfile, Loader=yaml.FullLoader)
        
        working_days = [nurse['working_days'] for nurse in data['nurses']]
        effort_level = [nurse['effort_level'] for nurse in data['nurses']]
        workload = [day['workload'] for day in data['days']]
    
    if working_days and effort_level and workload and len(working_days) == len(effort_level):
        num_nurses = len(working_days)
        num_days = len(workload)
        print('Data loaded successfully: {} nurses, {} days.'.format(num_nurses, num_days))
    else:
        raise Exception('WARNING: Data could not be loaded.')
        
    return working_days, effort_level, workload

[Back to the top](#top)

<a id="ns1_model_preparation"></a>
### 2.2. Preparing the model

To fully specify our model, first we need to initialize the appropriate class. In the following sections, we will fully implement the methods described down below, that add each one of the parts that take place in the final model.

In [81]:
def build_model(working_days, effort_level, workload,
                soft_nurse_lagrange_multiplier=1.0,
                hard_shift_lagrange_multiplier=1.0,
                consecutive_days_penalty=1.0):
    """
    Initialize and set up the binary quadratic model.
    
    :param working_days: List containing the number of working days agreed with each
        nurse, for every nurse (len(working_days) == num_nurses). E.g: [1, 1, 2, 2].
    :param effort_level: List containing the effort level that each nurse can deliver,
        to try to reach the total workload required for each day. E.g: [1, 4, 2, 3].
    :param workload: List containing the workload associated to each day, for every
        available day (len(workload) == num_days). E.g.: [5, 5, 5].
        
    :return: Binary Quadratic Model, ready to be provided to a solver.
    """
    
    print('\nBuilding Binary Quadratic Model...')
    
    # Get main parameters:
    num_nurses = len(working_days)
    num_days = len(workload)
    assert num_nurses == len(effort_level)

    # Initialize model.
    bqm = dimod.BinaryQuadraticModel(dimod.BINARY)
    for nurse in range(num_nurses):
        for day in range(num_days):
            bqm.add_variable((nurse, day), 0.0)  # Add variable x_{n,d}.
    
    # Constraint 1: try to adjust the number of working days for each nurse to the
    # desired value agreed with that nurse.
    add_soft_nurse_constraint(bqm, working_days, num_days,
                              soft_nurse_lagrange_multiplier=soft_nurse_lagrange_multiplier)

    # Constraint 2: ensure that the workload for each day is handled by one or several
    # nurses whose effort levels add up to that workload.
    add_hard_shift_constraint(bqm, effort_level, workload,
                              hard_shift_lagrange_multiplier=hard_shift_lagrange_multiplier)
    
    # Constraint 3: ensure that no nurse is assigned to work two days in a row.
    add_hard_nurse_constraint(bqm, num_nurses, num_days,
                              consecutive_days_penalty=consecutive_days_penalty)

    return bqm

[Back to the top](#top)

<a id="ns1_soft_nurse_constraint"></a>
### 2.3. Adding the soft nurse constraint

Our solutions should try to comply with the number of workdays previously agreed with each nurse. As expressed below, for each nurse, if we add up all the binary variables associated to that nurse (as they either have a value of 1 if the nurse has been assigned to a specific day, and 0 otherwise), the final value should be equal to that nurse's desired total number of workdays:

$$\forall n \in \{0, 1, ..., N-1\} \rightarrow \sum \limits _{d = 0}^{D - 1}  x_{n,d} = F_n$$

We can transform this equality to a minimization by making sure that the configurations that complied with the previous equality are later associated to each of the global minima. This can be done by moving the $F_n$ to the left part of the equality and, knowing that the value of that expression is 0 when a solution is reached, we can square everything so that every other configuration (incorrect solutions) have an associated value greater than any of the correct solutions.

$$\forall n \in \{0, 1, ..., N-1\} \rightarrow argmin_x (\sum \limits _{d = 0}^{D - 1} x_{n,d} - F_n)²$$

Then, we extend this definition to consider all nurses, while still looking for the configuration that yields the minimum value:

$$argmin_x \sum \limits _{n = 0}^{N - 1} (\sum \limits _{d = 0}^{D - 1} x_{n,d} - F_n)²$$

We would need to transform this constraint into another one that can be implemented by providing each coefficient individually to the SDK, but that process can be a bit complex and will be covered when implementing the second constraint. For now, we can just use the method https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/generated/dimod.generators.combinations.html, that generates a BQM with the following shape (as obtained from the documentation):

$$(\sum \limits _{i}^{} x_i - k)²$$

It is very similar to the one at hand, but please note that the notation used in this last expression **does not match the one used during the rest of this notebook**. Therefore, $i$ and $k$ in that expression have nothing to do with our problem, and have to be mapped to our variables. If you look at the Hamiltonian that defines this constraint, and the expression we just discussed, you will notice that we can consider $k = F_n$, and $x_i$ will be mapped to a variable $x_{n,d}$ **for a specific day**, as the expression found in Ocean's documentation does not offer a double sum, so $i = d$ but we will have to iterate through the different values of $n$ (the nurses) ourselves.

We just need to use this method for each possible nurse (to account for the outer sum of our Hamiltonian). When using the method itself, provide the $n$ available values of $i$ as a list of the tuples ```(nurse, day)``` that are being considered for a fixed day, and the $k = F_n$ number of options (```working_days[nurse]```) that could be picked by the solver. The parameter ```soft_nurse_lagrange_multiplier``` can be used as the strength of the constraint.

In [82]:
def add_soft_nurse_constraint(bqm, working_days, num_days, soft_nurse_lagrange_multiplier=1.0):
    """
    This function adds the first constraint to the Binary Quadratic Model, modifying
    it in place.
    
    :param bqm: Binary Quadratic Model, ready to be provided to a solver.
    :param working_days: List containing the number of working days agreed with each
        nurse, for every nurse (len(working_days) == num_nurses). E.g: [1, 1, 2, 2].
    :param num_days: Total number of days for the nurse scheduling problem.
    :param lagrange_multiplier: Lagrange multiplier for this constraint.
    """
    
    # --------------------------------------------------
    # ADD YOUR CODE HERE
    # \/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
    
    num_nurses = len(working_days)
    pref = 1
    aux_bqm = dimod.BinaryQuadraticModel(dimod.BINARY)
    
    # Diagonal terms in soft nurse constraint, without the min_duty_days**2 term
    for nurse in range(num_nurses):
        for day in range(num_days):
            aux_bqm.add_variable((nurse, day), 0.0)
            index = nurse * num_days + day
            bias1 = soft_nurse_lagrange_multiplier * (pref ** 2 - (2 * int(num_days/num_nurses) * pref))
            if (nurse!=day):
                bqm.add_interaction(nurse, day, bias1)

    # Off-diagonal terms in soft nurse constraint include only the same nurse, across days
    for nurse in range(num_nurses):
        for day1 in range(num_days):
            for day2 in range(day1 + 1, num_days):

                index1 = nurse * num_days + day1
                index2 = nurse * num_days + day2
                bias2 = 2 * soft_nurse_lagrange_multiplier * pref ** 2
                bqm.add_interaction(index1, index2, bias2)
    
    
    # /\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
    # --------------------------------------------------
        
    print('   Soft nurse constraint added.')

[Back to the top](#top)

<a id="ns1_hard_shift_constraint"></a>
### 2.4. Adding the hard shift constraint

The second constraint tries to ensure that the workload expected for each day will be handled by one or more nurses whose combined effort levels add up to that day's requirement. For each day, the sum of the effort levels of the nurses assigned to it should be equal to that day's workload:

$$\forall d \in \{0, 1, ..., D-1\} \rightarrow \sum \limits _{n = 0}^{N - 1}  E_n x_{n,d} = W_d$$

We can transform this equality to a minimization expression just as we did in the previous section. But this time, we will go through the entire process of reaching a version of the constraint that explicitly defines the value of each coefficient, so that we can implement it at a lower level:

$$\forall d \in \{0, 1, ..., D-1\} \rightarrow argmin_x (\sum \limits _{n = 0}^{N - 1} E_n x_{n,d} - W_d)²$$

We extend this definition to consider all days, and operate to get the final expression, while still looking for the configuration that yields the minimum value:

$$argmin_x \sum \limits _{d = 0}^{D - 1} (\sum \limits _{n = 0}^{N - 1} E_n x_{n,d} - W_d)²$$
$$argmin_x \sum \limits _{d = 0}^{D - 1} ((\sum \limits _{n = 0}^{N - 1} E_n x_{n,d})² - 2 W_d \sum \limits _{n = 0}^{N - 1} E_n x_{n,d} + W_d²)$$
$$argmin_x \sum \limits _{d = 0}^{D - 1} ((\sum \limits _{m = 0}^{N - 1} E_m x_{m,d}) \times (\sum \limits _{n = 0}^{N - 1} E_n x_{n,d}) - 2 W_d \sum \limits _{n = 0}^{N - 1} E_n x_{n,d} + W_d²)$$
$$argmin_x \sum \limits _{d = 0}^{D - 1} ((E_n x_{n,d} + \sum \limits _{m = 0, m \neq n}^{N - 1} E_m x_{m,d}) \times (\sum \limits _{n = 0}^{N - 1} E_n x_{n,d}) - 2 W_d \sum \limits _{n = 0}^{N - 1} E_n x_{n,d} + W_d²)$$
$$argmin_x \sum \limits _{d = 0}^{D - 1} (\sum \limits _{n = 0}^{N - 1} (E_n x_{n,d})² + \sum \limits _{n = 0}^{N - 1} \sum \limits _{m = 0, m \neq n}^{N - 1} E_m E_n x_{m,d} x_{n,d} - 2 W_d \sum \limits _{n = 0}^{N - 1} E_n x_{n,d} + W_d²)$$

As every $x_{n,d}$ is a binary variable with a value of either 0 or 1, $x_{n,d}² = x_{n,d}$, and therefore the first term is actually a linear term that can be added together with the other linear term of the expression. We will also remove the constant value $W_d²$ as it does not affect the location of the global minimum. This will get us to the final expression that will be implemented:

$$argmin_x \sum \limits _{d = 0}^{D - 1} (\sum \limits _{n = 0}^{N - 1} E_n² x_{n,d} + \sum \limits _{n = 0}^{N - 1} \sum \limits _{m = 0, m \neq n}^{N - 1} E_m E_n x_{m,d} x_{n,d} - 2 W_d \sum \limits _{n = 0}^{N - 1} E_n x_{n,d} + W_d²)$$
$$argmin_x \sum \limits _{d = 0}^{D - 1} ((1 - 2 W_d) \sum \limits _{n = 0}^{N - 1} E_n x_{n,d} + \sum \limits _{n = 0}^{N - 1} \sum \limits _{m = 0, m \neq n}^{N - 1} E_m E_n x_{m,d} x_{n,d} + W_d²)$$
$$argmin_x \sum \limits _{d = 0}^{D - 1} ((1 - 2 W_d) \sum \limits _{n = 0}^{N - 1} E_n x_{n,d} + \sum \limits _{n = 0}^{N - 1} \sum \limits _{m = 0, m \neq n}^{N - 1} E_m E_n x_{m,d} x_{n,d})$$

To implement this last expression, you may use [BinaryQuadraticModel.add_variable](https://test-projecttemplate-dimod.readthedocs.io/en/latest/reference/bqm/generated/dimod.BinaryQuadraticModel.add_variable.html#dimod.BinaryQuadraticModel.add_variable) and [BinaryQuadraticModel.add_interaction](https://test-projecttemplate-dimod.readthedocs.io/en/latest/reference/bqm/generated/dimod.BinaryQuadraticModel.add_interaction.html#dimod.BinaryQuadraticModel.add_interaction). As we will define this constraint in an empty BQM, we can ignore all previous values of the coefficientes or our problem, and the new BQM ```aux_bqm``` will be added to the problem's Hamiltonian by using ```bqm.update(...)```. When using the two methods referenced, each parameter referencing an index (```u``` and ```v```) will be a tuple ```(nurse, day)```, referencing the specific variable(s) $x_{n,d}$ being affected. The bias will be coefficient associated to each linear o quadratic term. Remember to multiply all coefficients by the Lagrange multiplier for this constraint!

In [83]:
def add_hard_shift_constraint(bqm, effort_level, workload, hard_shift_lagrange_multiplier=1.0):
    """
    This function adds the second constraint to the Binary Quadratic Model, modifying
    it in place.
    
    :param bqm: Binary Quadratic Model, ready to be provided to a solver.
    :param effort_level: List containing the effort level that each nurse can deliver,
        to try to reach the total workload required for each day. E.g: [1, 4, 2, 3].
    :param workload: List containing the workload associated to each day, for every
        available day (len(workload) == num_days). E.g.: [5, 5, 5].
    :param lagrange_multiplier: Lagrange multiplier for this constraint.
    """
    
    # Get main parameters:
    num_nurses = len(effort_level)
    num_days = len(workload)
    
    aux_bqm = dimod.BinaryQuadraticModel(dimod.BINARY)
    # --------------------------------------------------
    # ADD YOUR CODE HERE
    # \/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/

    
    # Diagonal terms in hard shift constraint, without the num_nurses**2 term
    for nurse in range(num_nurses):
        for day in range(num_days):
            aux_bqm.add_variable((nurse, day), 0.0)
            index = nurse * num_days + day
           
            bias1 = hard_shift_lagrange_multiplier * (num_nurses ** 2 - (2 * num_days * num_nurses))
            if (nurse!=day):
                bqm.add_interaction(nurse, day, bias1)

    # Off-diagonal terms in hard shift constraint Include only the same day, across nurses
    for day in range(num_days):
        for nurse1 in range(num_nurses):
            for nurse2 in range(nurse1 + 1, num_nurses):
                index1 = nurse1 * num_days + day
                index2 = nurse2 * num_days + day
                bias2 = 2 * hard_shift_lagrange_multiplier * num_nurses ** 2
                bqm.add_interaction(index1, index2, bias2)
                
                
    # /\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
    # --------------------------------------------------
            
    bqm.update(aux_bqm)
        
    print('   Hard shift constraint added.')

[Back to the top](#top)

<a id="ns1_hard_nurse_constraint"></a>
### 2.5. Adding the hard nurse constraint

We will also want to avoid solutions where the same nurse works in two consecutive days. For this we will use a coefficient $J_{n,d,n',d'}$ (```penalty[nurse_1][nurse_2][day_1][day_2]```), that will act as a penalty when considering the same nurse ($n = n'$) over consecutive days ($d' = d + 1$).

Just as before, a new BQM is provided to insert the appropriate coefficients (in this case, just a bias equal to ```consecutive_days_penalty``` when two consecutive days are being considered for the same nurse), so that the new BQM can be added to the problem Hamiltonian right at the end of the method.

In [84]:
def add_hard_nurse_constraint(bqm, num_nurses, num_days, consecutive_days_penalty=1.0):
    """
    This function adds the third constraint to the Binary Quadratic Model, modifying
    it in place, applying a penalty when the same nurse is assigned to two days in a
    row.
    
    :param bqm: Binary Quadratic Model, ready to be provided to a solver.
    :param num_nurses: Total number of nurses for the nurse scheduling problem.
    :param num_days: Total number of days for the nurse scheduling problem.
    """

    # Initialize model.
    aux_bqm = dimod.BinaryQuadraticModel(dimod.BINARY)
    for nurse in range(num_nurses):
        for day in range(num_days):
            aux_bqm.add_variable((nurse, day), 0.0)  # Add variable x_{n,d}.
    
    # Apply constraint.
    # --------------------------------------------------
    # ADD YOUR CODE HERE
    # \/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
    
        nurse_day1 = nurse * num_days + day
        nurse_day2 = nurse * num_days + (day+1)
        bqm.add_interaction(nurse_day1, nurse_day2, consecutive_days_penalty)
    
    # /\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
    # --------------------------------------------------
            
    bqm.update(aux_bqm)
        
    print('   Hard nurse constraint added.')

[Back to the top](#top)

<a id="ns1_postprocessing"></a>
### 2.6. Post-processing

We should check our solution to ensure that it complies with all constraints and requirements from the problem.

In [85]:
def check_solution(solution, working_days, effort_level, workload):
    """
    Classically checks that nurse scheduling obtained from the solver complies with
    the given constraints.
    
    :param solution: Nurse scheduling, expressed as a list of tuples where each
        tuple references a nurse that works on a certain day. E.g.: if the solution
        were to be [(0, 0), (1, 0), (2, 1)], both nurses 0 and 1 will work on day 0,
        and nurse 2 would work on day 1.
    :param working_days: List containing the number of working days agreed with each
        nurse, for every nurse (len(working_days) == num_nurses). E.g: [1, 1, 2, 2].
    :param effort_level: List containing the effort level that each nurse can deliver,
        to try to reach the total workload required for each day. E.g: [1, 4, 2, 3].
    :param workload: List containing the workload associated to each day, for every
        available day (len(workload) == num_days). E.g.: [5, 5, 5].
    """
    
    # Get main parameters:
    num_nurses = len(working_days)
    num_days = len(workload)
    assert num_nurses == len(effort_level)
    
    check_soft_nurse_constraint(solution, num_nurses, working_days)
    check_hard_shift_constraint(solution, num_days, effort_level, workload)
    check_hard_nurse_constraint(solution, num_nurses)

In [86]:
def check_soft_nurse_constraint(solution, num_nurses, working_days):
    """
    Checks the first constraint of the nurse scheduling problem: each nurse should
    be assigned the desired number of workdays.
    
    :param solution: Nurse scheduling, expressed as a list of tuples where each
        tuple references a nurse that works on a certain day. E.g.: if the solution
        were to be [(0, 0), (1, 0), (2, 1)], both nurses 0 and 1 will work on day 0,
        and nurse 2 would work on day 1.
    :param num_nurses: Total number of nurses for the nurse scheduling problem.
    :param working_days: List containing the number of working days agreed with each
        nurse, for every nurse (len(working_days) == num_nurses). E.g: [1, 1, 2, 2].
    """
    
    print('\nChecking soft nurse constraint:')
    for nurse in range(num_nurses):
        assigned_working_days = sum(
            1
            for (assigned_nurse, assigned_day) in solution
            if nurse == assigned_nurse
        )
        print('   {} -> Nurse {} was expected to work {} days. Days assigned: {}.'.format(
            'ok!' if working_days[nurse] == assigned_working_days else 'FAIL',
            nurse, working_days[nurse], assigned_working_days
        ))

In [87]:
def check_hard_shift_constraint(solution, num_days, effort_level, workload):
    """
    Checks the second constraint of the nurse scheduling problem: the workload
    associated to each day should be handled by one or more nurses whose combined
    efforts add up to the overall effort level required.
    
    :param solution: Nurse scheduling, expressed as a list of tuples where each
        tuple references a nurse that works on a certain day. E.g.: if the solution
        were to be [(0, 0), (1, 0), (2, 1)], both nurses 0 and 1 will work on day 0,
        and nurse 2 would work on day 1.
    :param num_days: Total number of days for the nurse scheduling problem.
    :param effort_level: List containing the effort level that each nurse can deliver,
        to try to reach the total workload required for each day. E.g: [1, 4, 2, 3].
    :param workload: List containing the workload associated to each day, for every
        available day (len(workload) == num_days). E.g.: [5, 5, 5].
    """
    
    print('\nChecking hard shift constraint:')
    for day in range(num_days):
        combined_efforts = sum(
            effort_level[assigned_nurse]
            for (assigned_nurse, assigned_day) in solution
            if day == assigned_day
        )
        print('   {} -> Day {} is expected to have a workload of {}. Efforts assigned: {}.'.format(
            'ok!' if combined_efforts >= workload[day] else 'FAIL',  # '>' also valid.
            day, workload[day], combined_efforts
        ))

In [88]:
def check_hard_nurse_constraint(solution, num_nurses):
    """
    Checks the third constraint of the nurse scheduling problem: no nurse should be
    assigned to work two days in a row.
    
    :param solution: Nurse scheduling, expressed as a list of tuples where each
        tuple references a nurse that works on a certain day. E.g.: if the solution
        were to be [(0, 0), (1, 0), (2, 1)], both nurses 0 and 1 will work on day 0,
        and nurse 2 would work on day 1.
    :param num_nurses: Total number of nurses for the nurse scheduling problem.
    """
    
    print('\nChecking hard nurse constraint:')
    for nurse in range(num_nurses):
        violations = sum(
            1
            for (assigned_nurse, assigned_day) in solution
            if (assigned_nurse, assigned_day + 1) in solution
        )
        print('   {} -> Nurse {} should not work two consecutive days. Violations found: {}.'.format(
            'ok!' if violations == 0 else 'FAIL',
            nurse, violations
        ))

[Back to the top](#top)

<a id="ns1_execution"></a>
### 2.7. Execution and results

We can now run the entire algorithm from end to end.

In [89]:
working_days, effort_level, workload = get_problem_data(
    # Uncomment one of the following lines.
    
    'nurse_scheduling_small.yml'
    #'nurse_scheduling_medium.yml'
    #'nurse_scheduling_large.yml'
)
bqm = build_model(
    working_days, effort_level, workload,
    soft_nurse_lagrange_multiplier=1.0, hard_shift_lagrange_multiplier=1.0, consecutive_days_penalty=1.0
)

# Sample.
print('Running...')
num_reads = 100
sample_set = neal.SimulatedAnnealingSampler().sample(bqm, num_reads=num_reads)
#sample_set = dwave.system.LeapHybridSampler().sample(bqm)

# Sample and retrieve results.
best_sample = sample_set.aggregate().first
solution = [
    item
    for item, item_selected in best_sample.sample.items()
    if item_selected
]

# Check results clasically.
check_solution(solution, working_days, effort_level, workload)

# Display results.
print('\nSolution:')
for i in range(len(solution)):
    nurse, day = solution[i]
    print('   Nurse {} works on day {}.'.format(nurse, day))

Loading file: nurse_scheduling_small.yml
Data loaded successfully: 4 nurses, 3 days.

Building Binary Quadratic Model...
   Soft nurse constraint added.
   Hard shift constraint added.
   Hard nurse constraint added.
Running...

Solution:
   Nurse 0 works on day 1.
   Nurse 0 works on day 2.
   Nurse 1 works on day 0.
   Nurse 1 works on day 1.
   Nurse 2 works on day 2.
   Nurse 3 works on day 1.


TypeError: cannot unpack non-iterable int object

[Back to the top](#top)

***

<a id="ref"></a>

## 4. Sources and additional references:

- [Ikeda, K., Nakamura, Y. & Humble, T.S. Application of Quantum Annealing to Nurse Scheduling Problem. Sci Rep 9, 12837 (2019).](https://doi.org/10.1038/s41598-019-49172-3)